# Algorithms for Computation Biology #

#### Author: Roshan Lodha ####

This notebook is a demo I am using to rationalize and learn various algorithms in computational biology, including Needleman-Wunsch, its variants, and Smith-Waterman. Please do NOT use this for actual sequence alignments.

Future routes include implementing the BLOSUM subsitution matrix and generalizing to protein sequences. Also, comparing NW and SW to bruteforce would highlight the importance of these algorithms. 

In [1]:
import numpy as np
#function to generate random DNA sequences
def dnaseqgen(n, gc=.5):
    seq = ""
    for i in range(n):
        if np.random.uniform() <= gc:
            if np.random.uniform() <= 0.5:
                seq += 'C'
            else:
                seq += 'G'
        else:
            if np.random.uniform() <= 0.5:
                seq += 'T'
            else:
                seq += 'A'
    return seq

## 1. Needleman-Wunsch ##

A good starting point is the traditional Needleman-Wunsch algorithm with end-gap penalties.

In [2]:
#global parameters are defined below
gap_penalty = -2
match = 1
mismatch = -1
X = dnaseqgen(10) #the first sequence
Y = dnaseqgen(10) #the second sequence
print(X)
print(Y)

ATCCTCCACG
GCCCTGAGAC


### Step 1: Initialization ###

In [3]:
s = np.zeros(shape=(len(X)+1, len(Y)+1))
p = np.zeros(shape=(len(X), len(Y)))

for x in range(len(X)+1):
    s[x][0] = x*gap_penalty
    
for y in range(len(Y)+1):
    s[0][y] = y*gap_penalty
    
print(s)

[[  0.  -2.  -4.  -6.  -8. -10. -12. -14. -16. -18. -20.]
 [ -2.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ -4.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ -6.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ -8.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [-10.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [-12.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [-14.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [-16.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [-18.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [-20.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]


### Step 2: Recurssion ###

In [4]:
def sim(x, y):
    if X[x-1] == Y[y-1]:
        return match
    return mismatch

for x in range(1, len(X)+1):
    for y in range(1, len(Y)+1):
        diag = s[x-1][y-1] + sim(x, y)
        gap_x = s[x][y-1] + gap_penalty
        gap_y = s[x-1][y] + gap_penalty
        scores = [diag, gap_x, gap_y]
        s[x][y] = max(scores)
        p[x-1][y-1] = np.argmax(scores)

print(s)

[[  0.  -2.  -4.  -6.  -8. -10. -12. -14. -16. -18. -20.]
 [ -2.  -1.  -3.  -5.  -7.  -9. -11. -11. -13. -15. -17.]
 [ -4.  -3.  -2.  -4.  -6.  -6.  -8. -10. -12. -14. -16.]
 [ -6.  -5.  -2.  -1.  -3.  -5.  -7.  -9. -11. -13. -13.]
 [ -8.  -7.  -4.  -1.   0.  -2.  -4.  -6.  -8. -10. -12.]
 [-10.  -9.  -6.  -3.  -2.   1.  -1.  -3.  -5.  -7.  -9.]
 [-12. -11.  -8.  -5.  -2.  -1.   0.  -2.  -4.  -6.  -6.]
 [-14. -13. -10.  -7.  -4.  -3.  -2.  -1.  -3.  -5.  -5.]
 [-16. -15. -12.  -9.  -6.  -5.  -4.  -1.  -2.  -2.  -4.]
 [-18. -17. -14. -11.  -8.  -7.  -6.  -3.  -2.  -3.  -1.]
 [-20. -17. -16. -13. -10.  -9.  -6.  -5.  -2.  -3.  -3.]]


In [5]:
#This cell is to visualize the path generated by NW. Running it is optional.
legend = {0: '`', 1: '<', 2: '^'}
p_vis = []
for x in range(len(X)):
    p_vis.append([])
    for y in range(len(Y)):
        p_vis[x].append(legend[p[x][y]])

print(np.matrix(p_vis))

[['`' '`' '`' '`' '`' '`' '`' '<' '`' '<']
 ['`' '`' '`' '`' '`' '<' '<' '`' '`' '`']
 ['`' '`' '`' '`' '<' '`' '`' '`' '`' '`']
 ['`' '`' '`' '`' '<' '<' '<' '<' '<' '`']
 ['`' '^' '^' '`' '`' '<' '<' '<' '<' '<']
 ['`' '`' '`' '`' '^' '`' '`' '`' '`' '`']
 ['`' '`' '`' '`' '`' '`' '`' '`' '`' '`']
 ['`' '^' '^' '^' '`' '`' '`' '`' '`' '<']
 ['`' '`' '`' '`' '`' '`' '^' '`' '`' '`']
 ['`' '^' '^' '^' '`' '`' '^' '`' '`' '^']]


### Step 3: Traceback ###

In [6]:
def traceback(x_pos = len(X)-1, y_pos = len(Y)-1, X_prime="", Y_prime=""):
    if x_pos == 0 or y_pos == 0:
        X_prime += X[x_pos]
        Y_prime += Y[y_pos]
        return X_prime[::-1], Y_prime[::-1]
    elif p[x_pos][y_pos] == 0:
        X_prime += X[x_pos]
        Y_prime += Y[y_pos]
        return traceback(x_pos-1, y_pos-1, X_prime, Y_prime)
    elif p[x_pos][y_pos] == 1:
        X_prime += '-'
        Y_prime += Y[y_pos]
        return traceback(x_pos, y_pos-1, X_prime, Y_prime)
    elif p[x_pos][y_pos] == 2:
        X_prime += X[x_pos]
        Y_prime += '-'
        return traceback(x_pos-1, y_pos, X_prime, Y_prime)
    else:
        print("Error during traceback.")
        return
        
X_aligned, Y_aligned = traceback()
print(X_aligned)
print(Y_aligned)

ATCCT-CCACG
GCCCTGAGAC-


## 2. Smith-Waterman ##

In [7]:
#global parameters are defined below
gap_penalty = -2
match = 1
mismatch = -1
X = dnaseqgen(10)
Y = dnaseqgen(12, 0.4)
print(X)
print(Y)

GCTGGCGTGC
TCTATACCGGTT


### Step 1: Initialization ###

In [8]:
s = np.zeros(shape=(len(X)+1, len(Y)+1))
p = np.zeros(shape=(len(X), len(Y)))
print(s)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


### Step 2: Recursion ###

In [9]:
def sim(x, y):
    if X[x-1] == Y[y-1]:
        return match
    return mismatch

h = 0

for x in range(1, len(X)+1):
    for y in range(1, len(Y)+1):
        diag = s[x-1][y-1] + sim(x, y)
        gap_x = s[x][y-1] + gap_penalty
        gap_y = s[x-1][y] + gap_penalty
        scores = [diag, gap_x, gap_y, 0]
        s[x][y] = max(scores)
        p[x-1][y-1] = np.argmax(scores)

print(s)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 1. 0. 2. 0. 1. 0. 0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 2. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 1. 0. 0.]
 [0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 2. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]]


### Step 3: Traceback ###

In [10]:
def traceback(x_pos, y_pos, X_prime = "", Y_prime = ""):
    if p[x_pos][y_pos] == 3:
        return X_prime[::-1], Y_prime[::-1]
    elif p[x_pos][y_pos] == 0:
        X_prime += X[x_pos]
        Y_prime += Y[y_pos]
        return traceback(x_pos-1, y_pos-1, X_prime, Y_prime)
    elif p[x_pos][y_pos] == 1:
        X_prime += '-'
        Y_prime += Y[y_pos]
        return traceback(x_pos, y_pos-1, X_prime, Y_prime)
    elif p[x_pos][y_pos] == 2:
        X_prime += X[x_pos]
        Y_prime += '-'
        return traceback(x_pos-1, y_pos, X_prime, Y_prime)
    else:
        print("Error during traceback.")
        return
    
r = np.argmax(s)//(len(Y)+1)-1
c = np.argmax(s)%(len(Y)+1)-1
X_aligned, Y_aligned = traceback(r, c)
print(X_aligned)
print(Y_aligned)

CT
CT
